# Analysis of Benchmark Results for 500 Single-Shot HDF5 Files with Alcator C-Mod Data

This notebook analyzes performance when reading shot data from 500 HDF5 files. The files were created with the following properties:

* Each file holds single shot data for up to 61 signals. Some files have less signals because their MDSplus _dim_of_ data were not available or had quality issues.
* Cloud optimized using the paged aggregation file space management with the file page size of 8,000,000 bytes (8 MB).
* Size of each file is exactly 16,000,000 bytes (16 MB) due to the above file space management.
* The files have same HDF5 group hierarchy: `/shots/<SHOT_ID>/signals/<SIGNAL_NAME>`.
* The MDSplus `dim_of` data are stored as HDF5 dimension scale datasets in the `/shots/<SHOT_ID>` group and attached to the appropriate dimensions of the signal HDF5 datasets.
* Only unique dimension scales were stored by comparing the MD5 checksum of dimension scale values. This means there could be multiple signals that share the same dimension scale.
* HDF5 datasets (dimension scales and signals) with total size greater than 8 kB are compressed with the deflate (zlib) compression at level 4.

## Benchmarks

All benchmarks were run on an AWS EC2 instance `m5.4xlarge` with 16 vCPU (8 CPU cores, 2 threads per core) and 64 GB memory. The benchmark cases were created by combining the following parameters:

* 500 HDF5 files.
* The files were stored either in the local file system (EBS) or S3.
* Data reading tasks were shared among 1, 2, 4, 8, 16, 24, 32, 48, or 64 Dask workers.
* HDF5 library page cache size was zero (off, for local files only) or 256 MB (note: holds entire file).
* Two data reading strategies:
  1. Read all the signals from one file. The file's signals are equally distributed across Dask workers.
  1. Read all signals from all the files, one signal at a time. The files to read a specific signal from are equally distributed accross Dask workers.

HDF5 library's Read-Only S3 (ROS3) driver was used to read data from the files in S3.

## TL;DR Conclusions

Given the benchmark results are so different for the two data access patterns -- reading all signals from one shot file and reading all signals from all shot files -- they are treated separately. Very important to note that all the shot files fit the ROS3 driver's cache of 16 MiB, which means that entire shot file was effectively downloaded to memory in just two S3 requests. This is nearly ideal situation so the benchmark results here are close to as good as possible.

For reading all signals from one shot file:

* A local file:
  * Runtime results in the 0.1 to 0.85 seconds range.
  * Best performance was with 8 Dask workers.
  * Noticeable worsening of performance with file page cache only for 48 and 64 Dask workers.
  * Modest improvement of performance ratio overall (max. 2) compared to using just one Dask worker. Using more than 24 Dask workers was worse than just one, which was likely caused by the EC2 instance's number of vCPUs.
* An S3 file:
  * Runtime results in the 0.37 to 1.6 seconds range.
  * Best performance was with 4 Dask workers.
  * Only 1.4 times max performance ratio gain compared to just one Dask worker. Using 16 or more Dask workers yielded worse performance than just one worker.

For reading all signals from all shot files:

* Benchmarks with less than 8 Dask workers were skipped because they were very slow for files in S3 in initial testing and therefore could not represent realistic use cases.
* Local files
  * Runtime results in the 12 to 81 seconds range.
  * Best performance was with 16 Dask workers (same number as the vCPUs).
  * Noticeably worse performance with file page cache. The results were always worse than a single Dask worker without file page cache.
  * Modest improvement of performance ratio overall (max. 2.4) compared to using just one Dask worker. Using more than 16 Dask workers yielded worsening performance, which was likely influenced by the vCPU number.
* S3 files
  * Runtime results in the 597 to 1117 seconds range.
  * Very similar best performance (ratio of 1.8) was achieved when the number of Dask workers ranged from 16 to 48. The performance with 64 workers was almost the same as when using just one, likely influenced by resource contention.

Taking all of the above into consideration:
* Reading all signals for a single shot isn't typically common, but it's fast enough even without multiple workers, whether the shot file is local or stored in S3.
* Reading all signals from all available shot files is a probable use case, especially when preparing analysis-ready individual signal files. The results benefited from multiple Dask workers but were also sensitive when their number was higher, exhibiting effects of resource contention when their number was exceeding the number of vCPUs. Having an EC2 instance with a lot of vCPUs only to repeatedly read many signals from shot files quickly does not seem justified. Much better approach is to create analysis-ready signal files for more efficient data access with cheaper EC2 instances.

---

## Benchmark Data Analysis

In [1]:
import pandas as pd
import hvplot.pandas  # noqa: F401
import holoviews as hv

pd.set_option("display.max_rows", None)
pd.set_option("display.max_columns", None)

hv.extension("bokeh")

Read benchmark data for the HDF5 files in the local file system:

In [2]:
lc_data = pd.read_csv("./ec2-local.csv")
lc_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 23312 entries, 0 to 23311
Data columns (total 12 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   worker#              23312 non-null  int64  
 1   obj-id               23312 non-null  object 
 2   open+read-data-time  23312 non-null  float64
 3   wrkr-num-objs        23312 non-null  int64  
 4   mean-obj-time        23312 non-null  float64
 5   num-dsets            23312 non-null  int64  
 6   mean-dset-time       23312 non-null  float64
 7   pb-size              23312 non-null  int64  
 8   num-workers          23312 non-null  int64  
 9   obj-type             23312 non-null  object 
 10  tot-num-obj          23312 non-null  int64  
 11  total-runtime        23312 non-null  float64
dtypes: float64(4), int64(6), object(2)
memory usage: 2.1+ MB


Read benchmark data for the HDF5 files in S3:

In [3]:
s3_data = pd.read_csv("./ec2-s3.csv")
s3_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 11655 entries, 0 to 11654
Data columns (total 12 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   worker#              11655 non-null  int64  
 1   obj-id               11655 non-null  object 
 2   open+read-data-time  11655 non-null  float64
 3   wrkr-num-objs        11655 non-null  int64  
 4   mean-obj-time        11655 non-null  float64
 5   num-dsets            11655 non-null  int64  
 6   mean-dset-time       11655 non-null  float64
 7   pb-size              11655 non-null  int64  
 8   num-workers          11655 non-null  int64  
 9   obj-type             11655 non-null  object 
 10  tot-num-obj          11655 non-null  int64  
 11  total-runtime        11655 non-null  float64
dtypes: float64(4), int64(6), object(2)
memory usage: 1.1+ MB


Page cache sizes used in the benchmarks:

In [4]:
lc_data["pb-size"].unique()

array([        0, 268435456])

In [5]:
s3_data["pb-size"].unique()

array([268435456])

Replace:
* Page cache sizes with simpler values.
* Correct a data error of reporting 500 for the total number of objects when reading by shot.

In [6]:
lc_data.replace(
    {
        "pb-size": {0: "off", 268435456: "264MB"},
        "tot-num-obj": {500: 1},
    },
    inplace=True,
)

s3_data.replace(
    {
        "pb-size": {0: "off", 268435456: "264MB"},
        "tot-num-obj": {500: 1},
    },
    inplace=True,
)

These were the benchmark parameter combinations:

In [7]:
lc_data[["obj-type", "pb-size", "num-workers"]].drop_duplicates()

,obj-type,pb-size,num-workers
0,shots,off,1
1,shots,off,2
3,shots,off,4
7,signals,off,8
495,shots,off,8
503,signals,off,16
1477,shots,off,16
1492,signals,off,24
2953,shots,off,24
2973,signals,off,32


In [8]:
s3_data[["obj-type", "pb-size", "num-workers"]].drop_duplicates()

,obj-type,pb-size,num-workers
0,shots,264MB,1
1,shots,264MB,2
3,shots,264MB,4
7,signals,264MB,8
495,shots,264MB,8
503,signals,264MB,16
1477,shots,264MB,16
1492,signals,264MB,24
2953,shots,264MB,24
2973,signals,264MB,32


Column `obj-type` describes data access type during one benchmark run:

* `obj-type = shots` means all signals from one shot file were read.
* `obj-type = signals` means all signals from all the files were read, one at a time.


Since the two ways of reading data by `shots` and `signals` are so different they cannot be compared to each other. Separate them into different DataFrames.

In [9]:
lc_shots = lc_data[lc_data["obj-type"] == "shots"]
s3_shots = s3_data[s3_data["obj-type"] == "shots"]
lc_signals = lc_data[lc_data["obj-type"] == "signals"]
s3_signals = s3_data[s3_data["obj-type"] == "signals"]

### Total Runtime and Peformance

Total benchmark runtime in the `tot-runtime` column is the elapsed time of the entire benchmark as measured by the main process. The total runtime encompasses:
1. Dividing data access plan across Dask workers and their intialization.
1. All Dask workers completing their jobs.
1. Collecting Dask worker benchmark data.

Below are four DataFrames with total runtimes separated for the signal and shot benchmarks. Their runtimes are so different that there is no point comparing them together. The new DataFrames include several original columns plus a new column `norm-tot-runtime`. This column holds computed performance ratios to the _baseline_ benchmark. The baseline benchmark is one of the available benchmarks selected because it represents the most common set of libhdf5 settings, compute resources, and data access. The baseline benchmarks for the two file location cases are:

* Local files:
    * Reading all signals for a shot: 1 Dask worker, no file page cache
    * Reading all signals for all the shots: 1 Dask worker, no file page cache
* S3 files:
    * Reading all signals for a shot: 1 Dask worker, 264 MB file page cache
    * Reading all signals for all the shots: 1 Dask worker, 264 MB file page cache

In [10]:
lc_shots_runtime = lc_shots[
    ["pb-size", "num-workers", "total-runtime"]
].drop_duplicates(ignore_index=True)
lc_shots_runtime["where"] = "Local"
lc_shots_runtime["norm-tot-runtime"] = (
    lc_shots_runtime.loc[0, "total-runtime"] / lc_shots_runtime["total-runtime"]
)

lc_signals_runtime = lc_signals[
    ["pb-size", "num-workers", "total-runtime"]
].drop_duplicates(ignore_index=True)
lc_signals_runtime["where"] = "Local"
lc_signals_runtime["norm-tot-runtime"] = (
    lc_signals_runtime.loc[0, "total-runtime"] / lc_signals_runtime["total-runtime"]
)

s3_shots_runtime = s3_shots[
    ["pb-size", "num-workers", "total-runtime"]
].drop_duplicates(ignore_index=True)
s3_shots_runtime["where"] = "S3"
s3_shots_runtime["norm-tot-runtime"] = (
    s3_shots_runtime.loc[0, "total-runtime"] / s3_shots_runtime["total-runtime"]
)

s3_signals_runtime = s3_signals[
    ["pb-size", "num-workers", "total-runtime"]
].drop_duplicates(ignore_index=True)
s3_signals_runtime["where"] = "S3"
s3_signals_runtime["norm-tot-runtime"] = (
    s3_signals_runtime.loc[0, "total-runtime"] / s3_signals_runtime["total-runtime"]
)

### Reading All Data from a Single Shot File

Plots of performance ratio and runtime when reading all data from a single local or S3 shot file:

In [11]:
plot_kwargs = {
    "x": "num-workers",
    "by": ["pb-size"],
}
(
    lc_shots_runtime.hvplot.line(y="norm-tot-runtime", **plot_kwargs)
    * lc_shots_runtime.hvplot.scatter(y="norm-tot-runtime", **plot_kwargs)
    * hv.HLine(1).opts(line_width=0.7, color="pink")
).options(
    legend_position="top_right",
    title="Local files performance (>1 better)",
    xlabel="Number of Dask workers",
    ylabel="Performance ratio",
    xlim=(0, lc_shots_runtime["num-workers"].max() + 1),
    ylim=(0, None),
    height=400,
    width=500,
) + (
    lc_shots_runtime.hvplot.line(y="total-runtime", **plot_kwargs)
    * lc_shots_runtime.hvplot.scatter(y="total-runtime", **plot_kwargs)
).options(
    legend_position="bottom_right",
    title="Local files runtime",
    xlabel="Number of Dask workers",
    ylabel="Total runtime / [s]",
    xlim=(0, lc_shots_runtime["num-workers"].max() + 1),
    ylim=(0, None),
    show_grid=True,
    height=400,
    width=500,
)

:Layout
   .Overlay.I  :Overlay
      .NdOverlay.I  :NdOverlay   [pb-size]
         :Curve   [num-workers]   (norm-tot-runtime)
      .NdOverlay.II :NdOverlay   [pb-size]
         :Scatter   [num-workers]   (norm-tot-runtime)
      .HLine.I      :HLine   [x,y]
   .Overlay.II :Overlay
      .NdOverlay.I  :NdOverlay   [pb-size]
         :Curve   [num-workers]   (total-runtime)
      .NdOverlay.II :NdOverlay   [pb-size]
         :Scatter   [num-workers]   (total-runtime)

Benchmark performance for the local files are split into three distinct groups. The best performance is for the "off, shots, shot layout" and "off, shots, signal layout" benchmarks. The second group are the "64MB, shots, shot layout" and "264MB, shots, shot layout". The rest of the benchmarks are the slowest group. The impact of more Dask workers is almost linear up to 8 workers and then reduces for the first two groups. The third group shows much lesser improvement for additional workers.

In [12]:
plot_kwargs = {
    "x": "num-workers",
    "by": ["pb-size"],
}
(
    (
        s3_shots_runtime.hvplot.line(y="norm-tot-runtime", **plot_kwargs)
        * s3_shots_runtime.hvplot.scatter(y="norm-tot-runtime", **plot_kwargs)
        * hv.HLine(1).opts(line_width=0.7, color="pink")
    ).options(
        legend_position="top_right",
        title="S3 files performance (>1 better)",
        xlabel="Number of Dask workers",
        ylabel="Performance ratio",
        xlim=(0, s3_shots_runtime["num-workers"].max() + 1),
        ylim=(0, None),
        height=400,
        width=500,
    )
    + (
        s3_shots_runtime.hvplot.line(y="total-runtime", **plot_kwargs)
        * s3_shots_runtime.hvplot.scatter(y="total-runtime", **plot_kwargs)
    ).options(
        legend_position="bottom_right",
        title="S3 files runtime",
        xlabel="Number of Dask workers",
        ylabel="Total runtime / [s]",
        xlim=(0, s3_shots_runtime["num-workers"].max() + 1),
        ylim=(0, None),
        show_grid=True,
        height=400,
        width=500,
    )
)

:Layout
   .Overlay.I  :Overlay
      .NdOverlay.I  :NdOverlay   [pb-size]
         :Curve   [num-workers]   (norm-tot-runtime)
      .NdOverlay.II :NdOverlay   [pb-size]
         :Scatter   [num-workers]   (norm-tot-runtime)
      .HLine.I      :HLine   [x,y]
   .Overlay.II :Overlay
      .NdOverlay.I  :NdOverlay   [pb-size]
         :Curve   [num-workers]   (total-runtime)
      .NdOverlay.II :NdOverlay   [pb-size]
         :Scatter   [num-workers]   (total-runtime)

### Reading All Signals from All Shot Files

Plots of performance ratio and runtime when reading all signals from all shot files either in local file system or in S3:

In [13]:
plot_kwargs = {
    "x": "num-workers",
    "by": ["pb-size"],
}
(
    lc_signals_runtime.hvplot.line(y="norm-tot-runtime", **plot_kwargs)
    * lc_signals_runtime.hvplot.scatter(y="norm-tot-runtime", **plot_kwargs)
    * hv.HLine(1).opts(line_width=0.7, color="pink")
).options(
    legend_position="top_right",
    title="Local files performance (>1 better)",
    xlabel="Number of Dask workers",
    ylabel="Performance ratio",
    xlim=(0, lc_signals_runtime["num-workers"].max() + 1),
    ylim=(0, None),
    height=400,
    width=500,
) + (
    lc_signals_runtime.hvplot.line(y="total-runtime", **plot_kwargs)
    * lc_signals_runtime.hvplot.scatter(y="total-runtime", **plot_kwargs)
).options(
    legend_position="bottom_right",
    title="Local files runtime",
    xlabel="Number of Dask workers",
    ylabel="Total runtime / [s]",
    xlim=(0, lc_signals_runtime["num-workers"].max() + 1),
    ylim=(0, None),
    show_grid=True,
    height=400,
    width=500,
)

:Layout
   .Overlay.I  :Overlay
      .NdOverlay.I  :NdOverlay   [pb-size]
         :Curve   [num-workers]   (norm-tot-runtime)
      .NdOverlay.II :NdOverlay   [pb-size]
         :Scatter   [num-workers]   (norm-tot-runtime)
      .HLine.I      :HLine   [x,y]
   .Overlay.II :Overlay
      .NdOverlay.I  :NdOverlay   [pb-size]
         :Curve   [num-workers]   (total-runtime)
      .NdOverlay.II :NdOverlay   [pb-size]
         :Scatter   [num-workers]   (total-runtime)

In [14]:
plot_kwargs = {
    "x": "num-workers",
    "by": ["pb-size"],
}
(
    (
        s3_signals_runtime.hvplot.line(y="norm-tot-runtime", **plot_kwargs)
        * s3_signals_runtime.hvplot.scatter(y="norm-tot-runtime", **plot_kwargs)
        * hv.HLine(1).opts(line_width=0.7, color="pink")
    ).options(
        legend_position="bottom_right",
        title="S3 files performance (>1 better)",
        xlabel="Number of Dask workers",
        ylabel="Performance ratio",
        xlim=(0, s3_signals_runtime["num-workers"].max() + 1),
        ylim=(0, None),
        height=400,
        width=500,
    )
    + (
        s3_signals_runtime.hvplot.line(y="total-runtime", **plot_kwargs)
        * s3_signals_runtime.hvplot.scatter(y="total-runtime", **plot_kwargs)
    ).options(
        legend_position="bottom_right",
        title="S3 files runtime",
        xlabel="Number of Dask workers",
        ylabel="Total runtime / [s]",
        xlim=(0, s3_signals_runtime["num-workers"].max() + 1),
        ylim=(0, None),
        show_grid=True,
        height=400,
        width=500,
    )
)

:Layout
   .Overlay.I  :Overlay
      .NdOverlay.I  :NdOverlay   [pb-size]
         :Curve   [num-workers]   (norm-tot-runtime)
      .NdOverlay.II :NdOverlay   [pb-size]
         :Scatter   [num-workers]   (norm-tot-runtime)
      .HLine.I      :HLine   [x,y]
   .Overlay.II :Overlay
      .NdOverlay.I  :NdOverlay   [pb-size]
         :Curve   [num-workers]   (total-runtime)
      .NdOverlay.II :NdOverlay   [pb-size]
         :Scatter   [num-workers]   (total-runtime)

### Display Some Worker Mean Runtimes

The `mean-obj-time` column holds mean read times of _objects_ in a shot file. Which _object_ is read depends on the `obj-type` column, with the values `shots` or `signals`. The `s3_signals` DataFrame 

In [15]:
s3_signals.groupby(["pb-size", "num-workers"])["mean-obj-time"].describe()

count      mean       std       min       25%       50%  \
pb-size num-workers                                                             
264MB   8             488.0  0.297264  0.022206  0.283973  0.290323  0.292609   
        16            974.0  0.322549  0.012753  0.286542  0.314425  0.320734   
        24           1461.0  0.472206  0.052201  0.287729  0.437843  0.469893   
        32           1947.0  0.627290  0.101379  0.285393  0.556155  0.623133   
        48           2801.0  0.943782  0.207328  0.298198  0.799741  0.931610   
        64           3815.0  2.171375  0.849501  0.287610  1.456049  2.139469   

                          75%       max  
pb-size num-workers                      
264MB   8            0.295944  0.491133  
        16           0.329100  0.398916  
        24           0.504216  0.705456  
        32           0.688505  1.162314  
        48           1.078346  1.933718  
        64           2.780532  5.314296

In [16]:
s3_signals.hvplot.box(
    y="mean-obj-time",
    by=["pb-size", "num-workers"],
).options(
    # ylim=(8, 11),
    title="S3 Files",
    height=400,
    show_legend=False,
    xlabel="File page buffer size, Number of Dask workers",
    ylabel="Worker mean signal read time / [seconds]",
    show_grid=True,
)

:BoxWhisker   [pb-size,num-workers]   (mean-obj-time)

In [ ]:
s3_signals.groupby(["obj-id", "pb-size", "num-workers"])["mean-obj-time"].describe()

count      mean       std       min  \
obj-id           pb-size num-workers                                        
aeqdsk_aminor    264MB   8              8.0  0.444262  0.061670  0.314572   
                         16            16.0  0.321849  0.012341  0.302672   
                         24            24.0  0.412288  0.095736  0.287729   
                         32            32.0  0.533910  0.157118  0.285393   
                         48            46.0  0.942895  0.201157  0.298198   
                         64            63.0  1.303327  0.400442  0.287610   
aeqdsk_area      264MB   8              8.0  0.305599  0.006782  0.296998   
                         16            16.0  0.320267  0.009527  0.304590   
                         24            24.0  0.467429  0.045929  0.404544   
                         32            32.0  0.635588  0.096515  0.465874   
                         48            46.0  0.944140  0.167864  0.679342   
                         64            63.0  1.212986  0.342134  0.577397   
aeqdsk_betan     264MB   8              8.0  0.298011  0.007092  0.290956   
                         16            16.0  0.321087  0.009688  0.302044   
                         24            24.0  0.472031  0.058461  0.367762   
                         32            32.0  0.634708  0.092454  0.504290   
                         48            46.0  0.957482  0.212084  0.506042   
                         64            63.0  1.256801  0.261377  0.639619   
aeqdsk_betap     264MB   8              8.0  0.295959  0.003773  0.288371   
                         16            16.0  0.324980  0.014894  0.305221   
                         24            24.0  0.477451  0.054522  0.395482   
                         32            32.0  0.634996  0.076173  0.508159   
                         48            46.0  0.954753  0.176721  0.532211   
                         64            63.0  1.300291  0.272546  0.719151   
aeqdsk_chisq     264MB   8              8.0  0.294863  0.003257  0.288102   
                         16            16.0  0.321158  0.009558  0.308821   
                         24            24.0  0.482880  0.065212  0.365506   
                         32            32.0  0.635254  0.087264  0.442611   
                         48            46.0  0.946589  0.163374  0.662113   
                         64            63.0  1.231946  0.275222  0.780699   
aeqdsk_doutl     264MB   8              8.0  0.293116  0.002294  0.290338   
                         16            16.0  0.322754  0.012739  0.306415   
                         24            24.0  0.461568  0.052873  0.351923   
                         32            32.0  0.616485  0.104447  0.433486   
                         48            46.0  0.935968  0.174865  0.592742   
                         64            63.0  1.317833  0.299243  0.692250   
aeqdsk_doutu     264MB   8              8.0  0.293195  0.001855  0.290637   
                         16            16.0  0.322581  0.010533  0.305295   
                         24            24.0  0.482417  0.060889  0.411198   
                         32            32.0  0.634978  0.087518  0.454933   
                         48            46.0  1.005698  0.257929  0.604387   
                         64            63.0  1.211349  0.345006  0.559048   
aeqdsk_eout      264MB   8              8.0  0.295885  0.002752  0.291667   
                         16            16.0  0.326291  0.024407  0.303684   
                         24            24.0  0.476438  0.059285  0.396604   
                         32            32.0  0.631153  0.089851  0.461692   
                         48            46.0  0.886779  0.206790  0.422868   
                         64            63.0  1.270153  0.224072  0.882237   
aeqdsk_kappa     264MB   8              8.0  0.292751  0.001846  0.290186   
                         16            16.0  0.319177  0.010739  0.304371   
                         24       

---